### Importing all the useful libraries

In [37]:
from pyspark.sql.functions import mean
from pyspark.sql.functions import when
from pyspark.sql import functions as F
import pyspark.sql.functions as f
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import GBTClassifier
from pyspark.mllib.evaluation import MulticlassMetrics
import matplotlib.pyplot as plt
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.util import MLUtils


In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

spark = SparkSession.builder.getOrCreate()

In [38]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [39]:
spark

### Performing some EDA before we pass the data

#### Redaing the file and also counting the number of null values in data


In [4]:
df = spark.read.csv('weatherAUS.csv',inferSchema =True, header = True)

In [5]:
for i in df.columns:
    count = df.filter(df[i] == 'NA').count()
    print('Total null counts for',i,'are:',count)

Total null counts for Date are: 0
Total null counts for Location are: 0
Total null counts for MinTemp are: 1485
Total null counts for MaxTemp are: 1261
Total null counts for Rainfall are: 3261
Total null counts for Evaporation are: 62790
Total null counts for Sunshine are: 69835
Total null counts for WindGustDir are: 10326
Total null counts for WindGustSpeed are: 10263
Total null counts for WindDir9am are: 10566
Total null counts for WindDir3pm are: 4228
Total null counts for WindSpeed9am are: 1767
Total null counts for WindSpeed3pm are: 3062
Total null counts for Humidity9am are: 2654
Total null counts for Humidity3pm are: 4507
Total null counts for Pressure9am are: 15065
Total null counts for Pressure3pm are: 15028
Total null counts for Cloud9am are: 55888
Total null counts for Cloud3pm are: 59358
Total null counts for Temp9am are: 1767
Total null counts for Temp3pm are: 3609
Total null counts for RainToday are: 3261
Total null counts for RainTomorrow are: 3267


#### I have dropped the columns which are independent and also the columns which had null values more than 60% 

In [6]:
df = df.drop('Date','Location','Evaporation','Sunshine','Cloud9am','Cloud3pm','Temp9am','Temp3pm')

In [7]:
numeric_col = [df['MinTemp'],df['MaxTemp'],df['Rainfall'],df['WindGustSpeed'],df['WindSpeed9am'],df['WindSpeed3pm'],df['Humidity9am'],df['Humidity3pm'],df['Pressure9am']]

#### Calculated the average so that I could Impute the null records.

In [8]:
for i in numeric_col:
  average = df.select([mean(i)]).show()
  average

+------------------+
|      avg(MinTemp)|
+------------------+
|12.194034380968894|
+------------------+

+------------------+
|      avg(MaxTemp)|
+------------------+
|23.221348275647014|
+------------------+

+------------------+
|     avg(Rainfall)|
+------------------+
|2.3609181499168272|
+------------------+

+------------------+
|avg(WindGustSpeed)|
+------------------+
| 40.03523007167319|
+------------------+

+------------------+
| avg(WindSpeed9am)|
+------------------+
|14.043425914971502|
+------------------+

+------------------+
| avg(WindSpeed3pm)|
+------------------+
|18.662656778887342|
+------------------+

+-----------------+
| avg(Humidity9am)|
+-----------------+
|68.88083133761887|
+-----------------+

+----------------+
|avg(Humidity3pm)|
+----------------+
|51.5391158755046|
+----------------+

+------------------+
|  avg(Pressure9am)|
+------------------+
|1017.6499397983058|
+------------------+



Updating the numeric column's null values with their respective mean 

In [9]:
updatedmint = df.withColumn("MinTemp",F.when(F.col("MinTemp") == 'NA',12.2).otherwise(F.col("MinTemp")))
updatedmaxt = updatedmint.withColumn("MaxTemp",F.when(F.col("MaxTemp") == 'NA',23.2).otherwise(F.col("MaxTemp")))
updatedrainf = updatedmaxt.withColumn("Rainfall",F.when(F.col("Rainfall") == 'NA',2.3).otherwise(F.col("Rainfall")))
updatedwind = updatedrainf.withColumn("WindGustSpeed",F.when(F.col("WindGustSpeed") == 'NA',40.0).otherwise(F.col("WindGustSpeed")))
updatedwinds9 = updatedwind.withColumn("WindSpeed9am",F.when(F.col("WindSpeed9am") == 'NA',14.0).otherwise(F.col("WindSpeed9am")))
updatedwinds3 = updatedwinds9.withColumn("WindSpeed3pm",F.when(F.col("WindSpeed3pm") == 'NA',18.6).otherwise(F.col("WindSpeed3pm")))
updatedhum9 = updatedwinds3.withColumn("Humidity9am",F.when(F.col("Humidity9am") == 'NA',68.8).otherwise(F.col("Humidity9am")))
updatedhum3 = updatedhum9.withColumn("Humidity3pm",F.when(F.col("Humidity3pm") == 'NA',51.5).otherwise(F.col("Humidity3pm")))
updatedpres9 = updatedhum3.withColumn("Pressure9am",F.when(F.col("Pressure9am") == 'NA',1017.7).otherwise(F.col("Pressure9am")))
inputted_numeric = updatedpres9.withColumn("Pressure3pm",F.when(F.col("Pressure3pm") == 'NA',1015.3).otherwise(F.col("Pressure3pm")))

In [10]:
updatedwindgust = inputted_numeric.withColumn("WindGustDir",F.when(F.col("WindGustDir") == 'NA','W').otherwise(F.col("WindGustDir")))
updatedwindir9 = updatedwindgust.withColumn("WindDir9am",F.when(F.col("WindDir9am") == 'NA','N').otherwise(F.col("WindDir9am")))
updatedwindir3 = updatedwindir9.withColumn("WindDir3pm",F.when(F.col("WindDir3pm") == 'NA','SE').otherwise(F.col("WindDir3pm")))
updatedraint = updatedwindir3.withColumn("RainToday",F.when(F.col("RainToday") == 'NA','No').otherwise(F.col("RainToday")))
inputted_weather = updatedraint.withColumn("RainTomorrow",F.when(F.col("RainTomorrow") == 'NA','No').otherwise(F.col("RainTomorrow")))

In [11]:
for i in inputted_weather.columns:
    count = inputted_weather.filter(inputted_weather[i] == 'NA').count()
    print('Total null counts for',i,'are:',count)

Total null counts for MinTemp are: 0
Total null counts for MaxTemp are: 0
Total null counts for Rainfall are: 0
Total null counts for WindGustDir are: 0
Total null counts for WindGustSpeed are: 0
Total null counts for WindDir9am are: 0
Total null counts for WindDir3pm are: 0
Total null counts for WindSpeed9am are: 0
Total null counts for WindSpeed3pm are: 0
Total null counts for Humidity9am are: 0
Total null counts for Humidity3pm are: 0
Total null counts for Pressure9am are: 0
Total null counts for Pressure3pm are: 0
Total null counts for RainToday are: 0
Total null counts for RainTomorrow are: 0


In [12]:
numeric_col = ['MinTemp','MaxTemp','Rainfall','WindGustSpeed','WindSpeed9am','WindSpeed3pm','Humidity9am','Humidity3pm','Pressure9am','Humidity3pm','Humidity9am','Pressure3pm']

In [13]:
for i in numeric_col:
    inputted_weather = inputted_weather.withColumn(i, inputted_weather[i].cast(DoubleType()))

#### Creating a pipeline and using StringIndexer and Onehotencoder 

In [14]:
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
 
from distutils.version import LooseVersion
 
categoricalColumns = ["WindGustDir","WindDir9am","WindDir3pm","RainToday","RainTomorrow"]
stages = [] # stages in Pipeline
for categoricalCol in categoricalColumns:
    # Category Indexing with StringIndexer
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    # Use OneHotEncoder to convert categorical variables into binary SparseVectors
    if LooseVersion(pyspark.__version__) < LooseVersion("3.0"):
        from pyspark.ml.feature import OneHotEncoderEstimator
        encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    else:
        from pyspark.ml.feature import OneHotEncoder
        encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]


#### Using vector assembler to join all the columns 

In [15]:
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numeric_col
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [16]:
stages

[StringIndexer_7a54230a9e19,
 OneHotEncoder_e46ddefc4825,
 StringIndexer_728dcc700763,
 OneHotEncoder_c7ffca6d6a0e,
 StringIndexer_a10bc86122ec,
 OneHotEncoder_def38810aeb2,
 StringIndexer_52c7f16a967e,
 OneHotEncoder_f1931ab329a2,
 StringIndexer_25bfad32c135,
 OneHotEncoder_1a703e664081,
 VectorAssembler_db8f9686015c]

### Defining Decision so that we can further add that in pipeine

In [17]:
dt = DecisionTreeClassifier(labelCol='RainTomorrowIndex' , featuresCol="features")

#### Listing all the parameters 

In [18]:
dtparamGrid = (ParamGridBuilder()
               .addGrid(dt.impurity, ["gini","entropy"])
             .addGrid(dt.maxDepth, [5])
             .addGrid(dt.maxBins, [5, 10, 15])
             .addGrid(dt.minInfoGain, [0.0, 0.2, 0.4])
             .build())

In [19]:
dtevaluator = BinaryClassificationEvaluator().setLabelCol("RainTomorrowIndex")

#### Defining the evaluator and number of folds for validation

In [20]:
dtcv = CrossValidator(estimator = dt,
                      estimatorParamMaps = dtparamGrid,
                      evaluator = dtevaluator,
                      numFolds = 4)

In [21]:
stages += [dtcv]

#### Splitting the data 

In [22]:
(trainingData, testData) = inputted_weather.randomSplit([0.8, 0.2], seed=12345)

#### Running the pipeline after assigning all the stages 

In [23]:

  
partialPipeline = Pipeline().setStages(stages)
pipelineModel = partialPipeline.fit(trainingData)
preppedDataDF = pipelineModel.transform(testData)

#### Here I wanted to extract the best parameter used, so in the pipelined I defined the stage where hyperparameter tuning was done and then tried to extract the best parameters.

In [25]:
model=pipelineModel.stages[-1].bestModel

#### Here we can see all the parameters in the best model selected,
- impurity    - "gini"
- maxDepth    - "5"
- maxBins     - "5"
- minInfoGain - "0.0"

<br>

We can also look at other parameters as well. 

In [26]:
model.extractParamMap()

{Param(parent='DecisionTreeClassifier_31886e9e93c6', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False,
 Param(parent='DecisionTreeClassifier_31886e9e93c6', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10,
 Param(parent='DecisionTreeClassifier_31886e9e93c6', name='featuresCol', doc='features column name.'): 'features',
 Param(parent='DecisionTreeClassifier_31886e9e93c6', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gin

In [28]:
preds = preppedDataDF.select('RainTomorrowIndex','probability').rdd.map(lambda row: (float(row['probability'][1]), float(row['RainTomorrowIndex'])))

In [30]:
metrics = BinaryClassificationMetrics(preds)


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Calculating the area under ROC and AUC score which are bulit in functions already.

In [31]:
print("Area under PR = %s" % metrics.areaUnderPR)

Area under PR = 1.0


In [32]:
print("Area under ROC = %s" % metrics.areaUnderROC)

Area under ROC = 1.0


In [36]:
metrics = CurveMetrics(preds)
metrics.get_curve('fMeasureByThreshold')
metrics.get_curve('precisionByThreshold')
metrics.get_curve('recallByThreshold')

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


[(1.0, 1.0), (0.0, 1.0)]

In [34]:
preppedDataDF.show()

+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+----------------+-------------------+---------------+------------------+---------------+------------------+--------------+-----------------+-----------------+--------------------+--------------------+-------------+-----------+----------+
|MinTemp|MaxTemp|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|RainToday|RainTomorrow|WindGustDirIndex|WindGustDirclassVec|WindDir9amIndex|WindDir9amclassVec|WindDir3pmIndex|WindDir3pmclassVec|RainTodayIndex|RainTodayclassVec|RainTomorrowIndex|RainTomorrowclassVec|            features|rawPrediction|probability|prediction|
+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+-----------